In [1]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from Custom_dataset import CustomDataset
from resnet1d import ResNet1D

In [5]:
kernel_size = 16
stride = 2
n_block = 48
downsample_gap = 6
increasefilter_gap = 12

In [6]:
model = ResNet1D(
    in_channels=12,
    base_filters=64, # 64 for ResNet1D, 352 for ResNeXt1D
    kernel_size=kernel_size,
    stride=stride, 
    groups=32, 
    n_block=n_block, 
    n_classes=1, 
    downsample_gap=downsample_gap,
    increasefilter_gap=increasefilter_gap, 
    use_do=True)

In [7]:
model

ResNet1D(
  (first_block_conv): MyConv1dPadSame(
    (conv): Conv1d(12, 64, kernel_size=(16,), stride=(1,))
  )
  (first_block_bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (first_block_relu): ReLU()
  (basicblock_list): ModuleList(
    (0): BasicBlock(
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (do1): Dropout(p=0.5, inplace=False)
      (conv1): MyConv1dPadSame(
        (conv): Conv1d(64, 64, kernel_size=(16,), stride=(1,), groups=32)
      )
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU()
      (do2): Dropout(p=0.5, inplace=False)
      (conv2): MyConv1dPadSame(
        (conv): Conv1d(64, 64, kernel_size=(16,), stride=(1,), groups=32)
      )
      (max_pool): MyMaxPool1dPadSame(
        (max_pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (1): Basic